In [1]:
# from nltk.corpus import movie_reviews, stopwords
# import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import cross_val_score
# from sklearn.metrics import roc_auc_score
from time import time
# import matplotlib.pyplot as plt
from os import listdir

import plotly.graph_objects as go

In [2]:
listdir()

['products_sentiment_train.tsv',
 'products_sentiment_sample_submission.csv',
 '.ipynb_checkpoints',
 'products_sentiment_test.tsv',
 'competition.ipynb',
 'to_kaggle.csv']

In [3]:
train = pd.read_csv('products_sentiment_train.tsv', sep = '\t', names = ['review', 'class'])
train.head()

review  class
0          2 . take around 10,000 640x480 pictures .      1
1  i downloaded a trial version of computer assoc...      1
2  the wrt54g plus the hga7t is a perfect solutio...      1
3  i dont especially like how music files are uns...      0
4  i was using the cheapie pail ... and it worked...      1

In [8]:
test = pd.read_csv('products_sentiment_test.tsv', sep = '\t')
test.head()

Id                                               text
0   0  so , why the small digital elph , rather than ...
1   1  3/4 way through the first disk we played on it...
2   2  better for the zen micro is outlook compatibil...
3   3    6 . play gameboy color games on it with goboy .
4   4  likewise , i 've heard norton 2004 professiona...

In [9]:
documents = train['review'].values
categories = train['class'].values

### Simple model

In [10]:
pipeline_cv = Pipeline([
    ('vect', CountVectorizer()),
    ('lr', LogisticRegression(solver = 'liblinear'))
])

pipeline_tfidf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('lr', LogisticRegression(solver = 'liblinear'))
])

In [12]:
score_cv = cross_val_score(pipeline_cv, train['review'], train['class'], cv = 5)
score_tfidf = cross_val_score(pipeline_tfidf, train['review'], train['class'], cv = 5)
score_cv, score_tfidf, score_cv.mean(), score_tfidf.mean() 

(array([0.775 , 0.77  , 0.77  , 0.7575, 0.7725]),
 array([0.77  , 0.7675, 0.7675, 0.7475, 0.775 ]),
 0.7689999999999999,
 0.7655000000000001)

In [13]:
classifiers = {
    'LogisticRegression': LogisticRegression(solver = 'liblinear', random_state=42),
    'LinearSVC': LinearSVC(random_state=42),
    'SGDClassifier': SGDClassifier(random_state=42)
}

for clf in classifiers:
    pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('clf', classifiers[clf])
    ])
    score = cross_val_score(pipeline, documents, categories, cv = 5)
    print(clf, ': ', score.mean())

LogisticRegression :  0.7689999999999999
LinearSVC :  0.754
SGDClassifier :  0.7455


In [17]:
classifiers = {
    'LogisticRegression': LogisticRegression(solver = 'liblinear', random_state=42),
    'LinearSVC': LinearSVC(random_state=42),
    'SGDClassifier': SGDClassifier(random_state=42)
}

for clf in classifiers:
    pipeline = Pipeline([
        ('vect', TfidfVectorizer()),
        ('clf', classifiers[clf])
    ])
    score = cross_val_score(pipeline, documents, categories, cv = 5)
    print(clf, ': ', score.mean())

LogisticRegression :  0.7655000000000001
LinearSVC :  0.7689999999999999
SGDClassifier :  0.7515


### Playing with min_df

In [14]:
df = pd.DataFrame(columns = ('min_df', 'accuracy_CV', 'accuracy_TFIDF'))
t = time()
for i in range(100):
    pipeline_CV = Pipeline([
        ('vect', CountVectorizer(min_df = i + 1, ngram_range = (1,2))),
        ('lr', LogisticRegression(solver = 'liblinear'))
    ])
    pipeline_TFIDF = Pipeline([
        ('vect', TfidfVectorizer(min_df = i + 1, ngram_range = (1,2))),
        ('lr', LogisticRegression(solver = 'liblinear'))
    ])
    score_CV = cross_val_score(pipeline_CV, documents, categories, cv = 5)
    score_TFIDF = cross_val_score(pipeline_TFIDF, documents, categories, cv = 5)
    
    
    step_df = pd.DataFrame({'min_df': [i + 1], 'accuracy_CV': score_CV.mean(), 'accuracy_TFIDF': score_TFIDF.mean()})
    df = df.append(step_df, ignore_index = True)
    if i % 10 == 9: print(i + 1,': ', round(time() - t, 2))
    

10 :  8.61
20 :  15.15
30 :  21.54
40 :  27.87
50 :  34.24
60 :  40.57
70 :  46.87
80 :  53.06
90 :  59.24
100 :  65.41


In [15]:
fig = go.Figure()
# Add traces
fig.add_trace(go.Scatter(
    x = df['min_df'], y = df['accuracy_CV'], mode='markers', name = 'Count Vectorizer')
)
fig.add_trace(go.Scatter(
    x = df['min_df'], y = df['accuracy_TFIDF'], mode = 'markers', name = 'TFIDF Vectorizer')
)
fig.show()

# Playing with ngrams & stopwords

In [25]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aleksandr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
classifiers = {
    'LogisticRegression': LogisticRegression(solver = 'liblinear', random_state=42),
    'LinearSVC': LinearSVC(random_state=42),
    'SGDClassifier': SGDClassifier(random_state=42)
}

for clf in classifiers:
    pipeline_ng = Pipeline([
        ('vect', CountVectorizer(ngram_range = (1,2))),
        ('clf', classifiers[clf])
    ])
    score = cross_val_score(pipeline_ng, documents, categories, cv = 5)
    print(clf, ': ', score.mean())

LogisticRegression :  0.7710000000000001
LinearSVC :  0.763
SGDClassifier :  0.7615


In [34]:
classifiers = {
    'LogisticRegression': LogisticRegression(solver = 'liblinear', random_state=42),
    'LinearSVC': LinearSVC(random_state=42),
    'SGDClassifier': SGDClassifier(random_state=42)
}

for clf in classifiers:
    pipeline_stp = Pipeline([
        ('vect', CountVectorizer(stop_words= stopwords.words('english'))),
        ('clf', classifiers[clf])
    ])
    score = cross_val_score(pipeline_stp, documents, categories, cv = 5)
    print(clf, ': ', score.mean())

LogisticRegression :  0.7515
LinearSVC :  0.7395
SGDClassifier :  0.7344999999999999


In [35]:
classifiers = {
    'LogisticRegression': LogisticRegression(solver = 'liblinear', random_state=42),
    'LinearSVC': LinearSVC(random_state=42),
    'SGDClassifier': SGDClassifier(random_state=42)
}

for clf in classifiers:
    pipeline_ng_stp = Pipeline([
        ('vect', CountVectorizer(ngram_range = (1,2), stop_words= stopwords.words('english'))),
        ('clf', classifiers[clf])
    ])
    score = cross_val_score(pipeline_ng_stp, documents, categories, cv = 5)
    print(clf, ': ', score.mean())

LogisticRegression :  0.75
LinearSVC :  0.7325
SGDClassifier :  0.729


## Create dataset for Kaggle

In [36]:
pipeline_ng.fit(train['review'], train['class'])

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf',
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                     

In [37]:
to_kaggle = pd.DataFrame({
    'Id': test['Id'],
    'y':pipeline_ng.predict(test['text'])
})

In [38]:
to_kaggle.to_csv('to_kaggle_2.csv', index = False)

# Screenshot

![title](screenshot.png)